# Converting 80-Column Observations to ADES Format

#### This tutorial demonstrates how to convert observations from the legacy 80-column (MPC1992) format to the ADES format, and how to add RA & Dec measurement uncertainties.

The [80-column format](https://www.minorplanetcenter.net/iau/info/OpticalObs.html) is the legacy format historically used for submitting astrometric observations to the MPC. The [ADES (Astrometry Data Exchange Standard)](https://www.minorplanetcenter.net/mpcops/documentation/ades/) is the current standard for observation submissions.

One important limitation of the 80-column format is that it **cannot represent RA & Dec measurement uncertainties** (`rmsRA` and `rmsDec`). These fields are supported by ADES and are increasingly important for orbit determination. After converting from 80-column to ADES format, you can add these uncertainty values as a separate step.

The [`iau-ades`](https://github.com/IAU-ADES/ADES-Master) Python package provides tools for:
 - Converting 80-column format to ADES XML (`mpc80coltoxml`)
 - Converting ADES XML to PSV format (`xmltopsv`)
 - Validating ADES XML files (`valsubmit`)

Further information can be found at:
 - https://www.minorplanetcenter.net/iau/info/OpticalObs.html
 - https://www.minorplanetcenter.net/mpcops/documentation/ades/
 - https://github.com/IAU-ADES/ADES-Master

# Install & Import Packages

Here we install the `iau-ades` package and import the packages used in this tutorial.

In [ ]:
!pip install -q iau-ades

In [ ]:
import tempfile
import atexit
import os
import xml.etree.ElementTree as ET
from lxml import etree as lxml_etree
from ades.mpc80coltoxml import mpc80coltoxml
from ades.xmltopsv import xmltopsv
from ades.valsubmit import valsubmit

# Sample 80-Column Data

The 80-column format uses fixed-width fields in exactly 80 characters per line. Each line encodes one observation, including the object designation, date/time, RA, Dec, magnitude, and observatory code.

Full documentation of the format can be found at https://www.minorplanetcenter.net/iau/info/OpticalObs.html

Here we create a sample file containing three observations of the same object from observatory 413 (Siding Spring Observatory).

In [ ]:
# Three sample 80-column observation lines (each exactly 80 characters)
obs80_lines = [
    "     K16Q99B  C2016 08 29.52259 13 57 33.13 -09 12 14.3          17.4 R      413\n",
    "     K16Q99B  C2016 08 30.51234 13 58 12.45 -09 10 22.1          17.6 R      413\n",
    "     K16Q99B  C2016 08 31.50567 13 59 01.78 -09 08 30.5          17.5 R      413\n",
]

# Write to a temporary file
with tempfile.NamedTemporaryFile(mode='w', suffix='.obs80', delete=False) as f:
    f.writelines(obs80_lines)
    obs80_path = f.name
    atexit.register(lambda path=f.name: os.unlink(path) if os.path.exists(path) else None)

print(f"Sample 80-column file: {obs80_path}")
print(f"Each line is {len(obs80_lines[0].rstrip())} characters wide.")
print()
for line in obs80_lines:
    print(line, end='')

# Step 1: Convert 80-Column to XML

The `mpc80coltoxml` function from the `iau-ades` package converts 80-column format observations to ADES XML.

**Note:** The raw output from `mpc80coltoxml` places `<optical>` elements directly under the `<ades>` root, without the `<obsBlock>`, `<obsContext>`, and `<obsData>` wrapper elements required by the ADES submission schema. It also includes metadata elements (`subFmt`, `precTime`, `precRA`, `precDec`) that are not valid in the submission schema. We will fix these issues in the next step.

In [ ]:
# Convert 80-column to XML
with tempfile.NamedTemporaryFile(suffix='.xml', delete=False) as f:
    raw_xml_path = f.name
    atexit.register(lambda path=f.name: os.unlink(path) if os.path.exists(path) else None)

mpc80coltoxml(obs80_path, raw_xml_path)

# Display the raw output
with open(raw_xml_path) as f:
    print(f.read())

# Step 2: Post-Process for Submission

The raw XML output needs to be restructured before it can pass ADES submission validation:

1. **Add `obsBlock` wrapper** containing `obsContext` (metadata about the observatory, submitter, telescope, observers) and `obsData` (the observations)
2. **Remove invalid elements** (`subFmt`, `precTime`, `precRA`, `precDec`) that are not part of the submission schema

Below we define a helper function that performs this restructuring. You will need to provide your own observatory, submitter, telescope, and observer details.

In [ ]:
def prepare_for_submission(raw_xml_path, output_path,
                           mpc_code, submitter_name, 
                           aperture, design, detector,
                           observer_names):
    """
    Restructure raw mpc80coltoxml output into valid ADES submission format.
    
    Parameters
    ----------
    raw_xml_path : str
        Path to the raw XML from mpc80coltoxml.
    output_path : str
        Path to write the restructured XML.
    mpc_code : str
        Observatory MPC code (e.g. '413').
    submitter_name : str
        Name of the submitter.
    aperture : str
        Telescope aperture in meters.
    design : str
        Telescope design (e.g. 'Reflector').
    detector : str
        Detector type (e.g. 'CCD').
    observer_names : list of str
        Names of observers/measurers.
    """
    tree = ET.parse(raw_xml_path)
    raw_root = tree.getroot()

    # Build new structure: <ades> -> <obsBlock> -> <obsContext> + <obsData>
    new_root = ET.Element('ades', version='2022')
    obsBlock = ET.SubElement(new_root, 'obsBlock')

    # Add obsContext
    obsContext = ET.SubElement(obsBlock, 'obsContext')
    observatory = ET.SubElement(obsContext, 'observatory')
    ET.SubElement(observatory, 'mpcCode').text = mpc_code
    submitter = ET.SubElement(obsContext, 'submitter')
    ET.SubElement(submitter, 'name').text = submitter_name
    telescope = ET.SubElement(obsContext, 'telescope')
    ET.SubElement(telescope, 'aperture').text = aperture
    ET.SubElement(telescope, 'design').text = design
    ET.SubElement(telescope, 'detector').text = detector
    observers_elem = ET.SubElement(obsContext, 'observers')
    for name in observer_names:
        ET.SubElement(observers_elem, 'name').text = name
    measurers_elem = ET.SubElement(obsContext, 'measurers')
    for name in observer_names:
        ET.SubElement(measurers_elem, 'name').text = name

    # Add obsData with cleaned optical elements
    obsData = ET.SubElement(obsBlock, 'obsData')
    remove_tags = {'subFmt', 'precTime', 'precRA', 'precDec'}
    for optical in raw_root.findall('optical'):
        for child in list(optical):
            if child.tag in remove_tags:
                optical.remove(child)
        obsData.append(optical)

    # Write with pretty formatting using lxml
    lxml_root = lxml_etree.fromstring(ET.tostring(new_root, encoding='unicode'))
    lxml_tree = lxml_etree.ElementTree(lxml_root)
    lxml_etree.indent(lxml_tree, space='  ')
    lxml_tree.write(output_path, xml_declaration=True, encoding='UTF-8', pretty_print=True)

In [ ]:
# Prepare the XML for submission
with tempfile.NamedTemporaryFile(suffix='.xml', delete=False) as f:
    prepared_xml_path = f.name
    atexit.register(lambda path=f.name: os.unlink(path) if os.path.exists(path) else None)

prepare_for_submission(
    raw_xml_path,
    prepared_xml_path,
    mpc_code='413',
    submitter_name='J. Smith',
    aperture='0.5',
    design='Reflector',
    detector='CCD',
    observer_names=['J. Smith']
)

with open(prepared_xml_path) as f:
    print(f.read())

# Step 3: Add RA & Dec Uncertainties

The 80-column format has no fields for RA and Dec measurement uncertainties. In ADES, these are represented by the `rmsRA` and `rmsDec` elements (in arcseconds), which capture the random component of the astrometric uncertainty (1-sigma).

After converting from 80-column format, you can add these values based on your knowledge of the measurement precision. The elements must be placed in a specific position within each `<optical>` element: after `<dec>` and before `<astCat>`.

Below we define a helper function that inserts `rmsRA` and `rmsDec` values into each observation.

In [ ]:
def add_uncertainties(input_xml_path, output_xml_path, rms_ra, rms_dec):
    """
    Add rmsRA and rmsDec elements to each optical observation in an ADES XML file.

    Parameters
    ----------
    input_xml_path : str
        Path to input ADES XML file.
    output_xml_path : str
        Path to write the updated XML file.
    rms_ra : str
        RA uncertainty in arcseconds (e.g. '0.25').
    rms_dec : str
        Dec uncertainty in arcseconds (e.g. '0.20').
    """
    tree = ET.parse(input_xml_path)
    root = tree.getroot()

    for optical in root.findall('.//optical'):
        # Find the <dec> element to insert after it
        children = list(optical)
        dec_idx = None
        for i, child in enumerate(children):
            if child.tag == 'dec':
                dec_idx = i
                break

        if dec_idx is not None:
            rmsRA_elem = ET.Element('rmsRA')
            rmsRA_elem.text = rms_ra
            rmsDec_elem = ET.Element('rmsDec')
            rmsDec_elem.text = rms_dec
            optical.insert(dec_idx + 1, rmsRA_elem)
            optical.insert(dec_idx + 2, rmsDec_elem)

    # Write with pretty formatting using lxml
    lxml_root = lxml_etree.fromstring(ET.tostring(root, encoding='unicode'))
    lxml_tree = lxml_etree.ElementTree(lxml_root)
    lxml_etree.indent(lxml_tree, space='  ')
    lxml_tree.write(output_xml_path, xml_declaration=True, encoding='UTF-8', pretty_print=True)

In [ ]:
# Add RA & Dec uncertainties (in arcseconds)
with tempfile.NamedTemporaryFile(suffix='.xml', delete=False) as f:
    final_xml_path = f.name
    atexit.register(lambda path=f.name: os.unlink(path) if os.path.exists(path) else None)

add_uncertainties(prepared_xml_path, final_xml_path, rms_ra='0.25', rms_dec='0.20')

with open(final_xml_path) as f:
    print(f.read())

# Step 4: Validate the Final XML

We can now validate the final XML file against the ADES submission schema using `valsubmit` to confirm everything is correct.

For more details on validation, see the [Validating ADES Observation Files](mpc_tutorial_ades_validation.ipynb) tutorial.

In [ ]:
valsubmit(final_xml_path)
with open('valsubmit.file') as f:
    print("Validation result:", f.read().strip())

# Step 5: Convert to PSV Format

ADES observations can also be represented in PSV (Pipe-Separated Values) format, which is more human-readable than XML. The `xmltopsv` function converts the final XML to PSV.

The PSV output will include the `rmsRA` and `rmsDec` columns that we added in Step 3.

In [ ]:
# Convert the final XML to PSV
with tempfile.NamedTemporaryFile(suffix='.psv', delete=False) as f:
    final_psv_path = f.name
    atexit.register(lambda path=f.name: os.unlink(path) if os.path.exists(path) else None)

xmltopsv(final_xml_path, final_psv_path)

with open(final_psv_path) as f:
    print(f.read())

# Summary

In this tutorial we demonstrated the full workflow for converting 80-column format observations to ADES:

1. **Convert** 80-column to raw XML using `mpc80coltoxml`
2. **Post-process** the XML to add the required `obsBlock`/`obsContext`/`obsData` structure and remove elements not valid in the submission schema
3. **Add RA & Dec uncertainties** (`rmsRA`, `rmsDec`) which cannot be represented in the 80-column format
4. **Validate** the result using `valsubmit` (see [Validating ADES Observation Files](mpc_tutorial_ades_validation.ipynb))
5. **Convert to PSV** format if desired using `xmltopsv`

For information on submitting your ADES files to the MPC, see the [Submit Observations](mpc_tutorial_api_submission_submission.ipynb) tutorial.